In [3]:
import pandas as pd
import numpy as np
import pickle

from scipy.sparse import load_npz, csr_matrix
from sklearn.neighbors import NearestNeighbors
from scripts.caracteristica import caracteristica as car

from datasketch import MinHash, MinHashLSH

In [4]:
def jaccard_distance_sparse(u, v):
    intersection = u.multiply(v).sum()
    union = u.maximum(v).sum()
    return 1.0 - intersection / union

In [5]:
n_perm = 128

def minhash_vector(sparse_vector:csr_matrix, num_perm:int=n_perm):
    minhash = MinHash(num_perm=num_perm)
    for index in sparse_vector.indices:
        minhash.update(str(index).encode('utf8'))
    return minhash

In [6]:
k = 2
th = 0.5
lsh = pickle.load(open(f'../Data/lsh/k{k}_th{th}.pkl', 'rb'))

In [8]:
caracteristica, tweets = car(k=k)

In [15]:
indice = 1007

m = minhash_vector(caracteristica[indice])
aprox = lsh.query(m)

original = tweets.iloc[indice]
print(f'{original["screen_name"] + ";".ljust(10)} {original["text"]}\n')

repeticiones = {}
for i in aprox:
    twt = tweets.iloc[i]
    repeticiones[twt.screen_name] = repeticiones.get(twt.screen_name, 0) + 1
    print( f'{(str(i) + ",").ljust(8)} {twt["screen_name"].strip().ljust(20)}:'.ljust(30), twt['text'] )


    # if twt['screen_name'] not in repeticiones:
    #     repeticiones[twt['screen_name']] = 1
    # else:
    #     repeticiones[twt['screen_name']] += 1

TalkParisLondon;          @hugo_gutierrez_ la cc es lo que impulsa la crisis economica desconfianza en invertir ante las inminentes expropiaciones se

2066,    fn19100             : @hugo_gutierrez_ la cc es lo que impulsa la crisis economica desconfianza en invertir ante las inminentes expropiaciones setenteras
12,      TalkParisLondon     : @hugo_gutierrez_ la cc es lo que impulsa la crisis economica desconfianza en invertir ante las inminentes expropiaciones se


Seleccionamos tweets de original y el usuario mas repetido.

In [8]:
mas_repetido = max(repeticiones)
tweets = pd.read_parquet('../Data/tweets.parquet')
par = []
for c, name in enumerate(tweets['screen_name']):
    if name == original['screen_name']:
        tweet = tweets['text'].iloc[c]
        par.append(f'{name}: {tweet}')
    elif name == mas_repetido:
        tweet = tweets['text'].iloc[c]
        par.append(f'{name}: {tweet}')

Funcion que calcula diferencia jaccard entre los tweets, se la pedi a chatgpt asique igual seria buena idea revisarla en mas detalle.

In [11]:
import textdistance
import heapq

def find_most_similar_pairs(strings, k=30):
    pairs = []
    heap = []
    
    for i in range(len(strings)):
        for j in range(i + 1, len(strings)):
            similarity_score = textdistance.jaccard(strings[i], strings[j])
            
            heapq.heappush(heap, (similarity_score, (strings[i], strings[j])))
            if len(heap) > k:
                heapq.heappop(heap)
    
    while heap:
        pairs.append(heapq.heappop(heap)[1])
    
    return pairs[::-1]

Calculamos los tweets mas similares, y mantenemos solamente los pares en donde haya un tweet de usuario original y otro del mas repetido.

In [12]:
similar_pairs = find_most_similar_pairs(par)
for pair in similar_pairs:
    og = 0
    repetido = 0
    for tweet in pair:
        if mas_repetido in tweet:
            repetido += 1
        elif original['screen_name'] in tweet:
            og += 1
    if og == 1 and repetido == 1:
        print(pair)

('ajsjah_adadj: @macar3na_m @carreragonzalo @tere_marinovic aqui les dejo su i r x ', 'vanipetu: @tere_marinovic una calurosa bienvenida a la cc en antofagasta ')
('ajsjah_adadj: @tere_marinovic una ministra infiltrada ', 'vanipetu: @tere_marinovic jajajajaj grande tere te adoro ')
('ajsjah_adadj: @tere_marinovic una ministra infiltrada ', 'vanipetu: @jaime_bassa cc mentirosaaaaaa')
('ajsjah_adadj: @macar3na_m @carreragonzalo @tere_marinovic aqui les dejo su i r x ', 'vanipetu: @tere_marinovic jajajajaj grande tere te adoro ')
('ajsjah_adadj: @macar3na_m @carreragonzalo @tere_marinovic aqui les dejo su i r x ', 'vanipetu: @germanjeshu @tere_marinovic aprendimos de los mejores destructores de la historia de nuestro pais ')
('ajsjah_adadj: @tere_marinovic una ministra infiltrada ', 'vanipetu: @tere_marinovic una calurosa bienvenida a la cc en antofagasta ')
('ajsjah_adadj: @macar3na_m @carreragonzalo @tere_marinovic aqui les dejo su i r x ', 'vanipetu: @patriota_1972 conosco ateos politi